In [ ]:
import jupyros as jr
import rospy
from nav_msgs.msg import Odometry
from my_assignment.msg import PosVelData
from my_assignment.srv import GoalCounter
from sensor_msgs.msg import LaserScan
import actionlib
import actionlib.msg
import assignment_2_2022.msg
import matplotlib.pyplot as plt
import tf
from tf.transformations import quaternion_matrix
import numpy as np
from matplotlib.animation import FuncAnimation
global goal
global client
global publisher
global msg
global n_reached
global n_not_reached

n_reached = 0
n_not_reached = 0
#initialize the node
rospy.init_node('nodeA')

Node A Publisher
==============

In [ ]:
#publisher that writes the data on the topic /posVelData
publisher = rospy.Publisher("/posVelData", PosVelData, queue_size = 10)

In [ ]:
def callback(sub_data):
    
    posVelData = PosVelData()
    
    posVelData.x = sub_data.pose.pose.position.x
    posVelData.y = sub_data.pose.pose.position.y
    posVelData.vel_x = sub_data.twist.twist.linear.x
    posVelData.vel_y = sub_data.twist.twist.linear.y
    
    publisher.publish(posVelData)

In [ ]:
#subscribe to the topic /odom
jr.subscribe("/odom", Odometry, callback)

Node A Client
===========

In [ ]:
#crate action client
client = actionlib.SimpleActionClient('/reaching_goal', assignment_2_2022.msg.PlanningAction)
#wait for server
client.wait_for_server()

Set goal
-----------

In [ ]:
import ipywidgets as widgets

goal = assignment_2_2022.msg.PlanningGoal()

x = widgets.FloatText(description = "x:")
y = widgets.FloatText(description = "y:")
status = widgets.Text(value = "Simulation started!", disabled = True)

button_start = widgets.Button(description = "Send goal!", button_style = "success")

def on_button_start_clicked(b):
    
    goal.target_pose.pose.position.x = x.value
    goal.target_pose.pose.position.y = y.value
    
    #update goal list
    goal_list.value = goal_list.value + "Goal x: " + str(x.value) + "  Goal y: " + str(y.value)
    
    #send the goal returned by set_goal()
    client.send_goal(goal)
    
    x.disabled = True
    y.disabled = True
    button_start.disabled = True
    button_stop.disabled = False
    status.value = "Reaching goal!"
    

button_start.on_click(on_button_start_clicked)

button_stop = widgets.Button(description = "Cancel goal!", disabled = True, button_style = "danger")

def on_button_stop_clicked(b):
    client.cancel_goal()
    
button_stop.on_click(on_button_stop_clicked)
        

widgets.HBox([widgets.VBox([x, y]), widgets.VBox([button_start, button_stop]), status])


In [ ]:
global reached_list
global not_reached_list

reached_list = [0]
not_reached_list = [0]

def update_goals_lists():
    global n_reached, n_not_reached
    reached_list.append(n_reached)
    not_reached_list.append(n_not_reached)

def checkGoalFeedback(data):
    global n_reached, n_not_reached
    if data.feedback.stat == "Target reached!":
        x.disabled = False
        y.disabled = False
        button_start.disabled = False
        button_stop.disabled = True
        status.value = "Goal reached!"
        goal_list.value = goal_list.value + "    Goal reached!\n"
        n_reached += 1
        update_goals_lists()
    elif data.feedback.stat == "Target cancelled!":
        x.disabled = False
        y.disabled = False
        button_start.disabled = False
        button_stop.disabled = True
        status.value = "Goal cancelled!"
        goal_list.value = goal_list.value + "    Goal cancelled!\n"
        n_not_reached += 1
        update_goals_lists()

In [ ]:
jr.subscribe("/reaching_goal/feedback", assignment_2_2022.msg.PlanningActionFeedback, checkGoalFeedback)

Data display
==========

Position and velocity of the robot
------------------------------------------------

In [ ]:
posx = widgets.FloatText(description = "Pos x:", disabled = True)
posy = widgets.FloatText(description = "Pos y:", disabled = True)
velx = widgets.FloatText(description = "Vel x:", disabled = True)
vely = widgets.FloatText(description = "Vel y:", disabled = True)
    
widgets.HBox([widgets.VBox([posx, posy]), widgets.VBox([velx, vely])])
    

In [ ]:
class Visualiser:
    def __init__(self):
        self.fig, self.ax = plt.subplots()
        self.ln, = plt.plot([], [], 'ro')
        self.x_data, self.y_data = [] , []
    def plot_init(self):
        self.ax.set_xlim(-10, 10)
        self.ax.set_ylim(-10, 10)
        return self.ln
    def vis_callback(self, data):
        self.y_data.append(data.y)
        self.x_data.append(data.x)
        posx.value = data.x
        posy.value = data.y
        velx.value = data.vel_x
        vely.value = data.vel_y
        
    def update_plot(self, frame):
        self.ln.set_data(self.x_data, self.y_data)
        return self.ln

In [ ]:
%matplotlib widget

vis = Visualiser()
sub = jr.subscribe('/posVelData', PosVelData, vis.vis_callback)

ani = FuncAnimation(vis.fig, vis.update_plot, init_func=vis.plot_init)
plt.show(block=True)

Goal list
------------

In [ ]:
goal_list = widgets.Textarea(value = "Goal list:\n", disabled = True)

display(goal_list)

Closest obstacle
-----------------------

In [ ]:
rmin = widgets.FloatText(description = "Distance:", disabled = True)
ang = widgets.FloatText(description = "Angle:", disabled = True)

def laserCallback(scan):
    min_range = 100
    angle = 100
    for at, x in enumerate(scan.ranges):
        if x < min_range and x > scan.range_min:
            min_range = x
            angle = scan.angle_min + scan.angle_increment * at
    
    rmin.value = min_range
    ang.value = angle
    
widgets.HBox([rmin, ang], description = "Closest obstacle")

In [ ]:
jr.subscribe('/scan', LaserScan, laserCallback)

Reached/Not-Reached Targets
--------------------------------------------

In [ ]:
button_update = widgets.Button(description = "Update graph!", button_style = "success")

plt.figure(2)
line1, = plt.plot(reached_list, label="Reached", color = 'green')
line2, = plt.plot(not_reached_list, label="Not-Reached", color = 'red')
# Create a legend for the first line.
first_legend = plt.legend(handles=[line1], loc='lower left')
# Add the legend manually to the current Axes.
plt.gca().add_artist(first_legend)
# Create another legend for the second line.
plt.legend(handles=[line2], loc='lower right')
plt.show()

def on_button_update_clicked(b):
    line1, = plt.plot(reached_list, label="Reached", color = 'green')
    line2, = plt.plot(not_reached_list, label="Not-Reached", color = 'red')
    # Create a legend for the first line.
    first_legend = plt.legend(handles=[line1], loc='lower left')
    # Add the legend manually to the current Axes.
    plt.gca().add_artist(first_legend)
    # Create another legend for the second line.
    plt.legend(handles=[line2], loc='lower right')
    plt.show()
    
button_update.on_click(on_button_update_clicked)
display(button_update)